In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import cobra

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

from src.gem.yeast8model import Yeast8Model
from src.calc.matrix import ArrayCollection
from src.calc.ablation import vget_ablation_ratio

# Convenience functions

In [ ]:
def get_random_coords(coords, num_samples):
    return coords[np.random.choice(coords.shape[0], num_samples, replace=False), :]


def coords_to_dict(coords, carbon_exch):
    """Convenience"""
    return {
        "exch_ids": [carbon_exch, "r_1654"],
        "exch_points": coords,
    }

# Initialise model

In [ ]:
glc_exch_rate = 16.89
wt = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
# Default: lots of glucose
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
print("Model initialised")

# Load PKL to compute ratios

In [ ]:
grid_filename = "ec_grid_" + "glc" + "_amm"
grid_filepath = "../data/interim/" + grid_filename + ".pkl"
with open(grid_filepath, "rb") as handle:
    ablation_result_array = pickle.load(handle)

# Picking, ablation, record fluxes

In [ ]:
carbon_exch = "r_1714"
saturation_glc = 8.6869
saturation_amm = 1.4848
x_axis = np.linspace(0, 2 * saturation_glc, 32)
y_axis = np.linspace(0, 2 * saturation_amm, 32)

In [ ]:
# Pick random points from a grid based on a mask,
# i.e. whether the ratio is greater than ('big') or less than ('small') one
ratio = ArrayCollection(vget_ablation_ratio(ablation_result_array), x_axis, y_axis)
ratio_array_mask = ratio.array > 1

x_coords, y_coords = np.meshgrid(x_axis, y_axis)
big_ratio_coords = np.column_stack(
    (x_coords[ratio_array_mask], y_coords[ratio_array_mask])
)
small_ratio_coords = np.column_stack(
    (x_coords[~ratio_array_mask], y_coords[~ratio_array_mask])
)

num_samples = 3

big_ratio_coords_random = get_random_coords(big_ratio_coords, num_samples)
small_ratio_coords_random = get_random_coords(small_ratio_coords, num_samples)

# Perform ablation and record fluxes
big_ablation_result_list = wt.usgfluxes_list(
    coords_to_dict(big_ratio_coords_random, carbon_exch)
)
small_ablation_result_list = wt.usgfluxes_list(
    coords_to_dict(small_ratio_coords_random, carbon_exch)
)
all_ablation_result_list = np.concatenate(
    (big_ablation_result_list, small_ablation_result_list)
)
print(f"Ablation done for {num_samples} in each category.")

# Adjust data variable dimensions
multicond_enz_use_array = np.concatenate(all_ablation_result_list)
multicond_enz_use_array.shape

# UMAP

In [ ]:
import umap

In [ ]:
reducer = umap.UMAP()

In [ ]:
scaled_array = scale(multicond_enz_use_array)

In [ ]:
embedding = reducer.fit_transform(scaled_array)

In [ ]:
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
)